In [22]:
!pip install neo4j openai langchain tiktoken

In [23]:
from openai import OpenAI
from langchain.graphs import Neo4jGraph
import os
from langchain.vectorstores.neo4j_vector import Neo4jVector
from langchain.embeddings.openai import OpenAIEmbeddings
from neo4j import GraphDatabase
import re

## DATABASE CONNEXION

In [24]:
from langchain.graphs import Neo4jGraph

url = ""
username =""
password = ""
graph = Neo4jGraph(
    url=url,
    username=username,
    password=password
)

In [26]:
os.environ['OPENAI_API_KEY'] = ""

client = OpenAI()

url = ""  # Replace with your Neo4j server URI
username = ""
password = ""

graph2 = Neo4jGraph(
    url=url,
    username=username,
    password=password
)

vector_index = Neo4jVector.from_existing_graph(
    OpenAIEmbeddings(),
    url=url,
    username=username,
    password=password,
    index_name='tasks',
    node_label="Content",
    text_node_properties=['contenu'],
    embedding_node_property='embedding',
)

## FONCTIONS

In [27]:
def schema_extraction(graph):
  schema_visualisation = graph.query("call db.schema.visualization")
  rel_properties = graph.query("call db.schema.relTypeProperties")
  node_properties = graph.query("call db.schema.nodeTypeProperties")



  assistant = "Voici le schéma du graphe: " + str(schema_visualisation) + "Voici les propriété des différentes relations: " + str(rel_properties) + "Voici les propriétés des différents noeuds :" + str(node_properties)
  return schema_visualisation, rel_properties, node_properties


In [28]:
def run_cypher_query(query, graph):
    result = graph.query(query)
    return list(result)


def voisin_reponses(question,vector_index, graph):
  response = vector_index.similarity_search(question)
  reponses = []
  for res in response:
      texte_recherche = res.page_content.split("contenu: ")[1]
      reponses.append(texte_recherche)
      cypher_query = f"MATCH (c:Content)-[:has_commun_entity]->(e:Content) WHERE c.contenu = '{texte_recherche}' RETURN DISTINCT e.contenu as contenu;"
      resultats = run_cypher_query(cypher_query, graph)
      for record in resultats[:10]:
          reponses.append(record["contenu"])
  return list(set(reponses))

In [30]:
question = "tell me about Dune: Part Two, and the cast of the film Spider-Man: Far From Home"

In [36]:
question = "give me the set of persons who wrote and produced the same movie."

In [ ]:
question = "what movie did Leonardo Dicaprio acted in ?"

## Test With TWO RESPONSES

In [32]:
def reponse_question(question,vector_index, client, graph):
  facts = voisin_reponses(question,vector_index, graph)
  assistant = "here's a set of facts: " + str(facts)
  prompt_template = """
    give me an answer to this question based only on the set of facts: {}.
    Some fo these facts may not contain information necessery to the answer, you can ignore them.
    Dont add information from your knowledge, if you dont have the information in the set of facts dont add answer from your own.
    """.format(question)

  messages = [
              {"role": "user", "content": prompt_template},
              {"role": "assistant", "content": assistant},
              ]

  response = client.chat.completions.create(
      model="gpt-3.5-turbo",
      messages=messages,
      temperature=0
  )
  print(response.choices[0].message.content)
  return response.choices[0].message.content

In [33]:
def questionAnswering(question, graph, client):
  ##EXTRACTION SCHEMA DU GRAPH
  schema_visualisation, rel_properties, node_properties = schema_extraction(graph)
  ##GENERATION DE LA REQUETE CYPHER
  query_result = ""
  assistant = "Here is the graph schema: " + str(schema_visualisation) + "\n Here is the properties of the relations of the graph: " + str(rel_properties) + "\n Here is the properties of the nodes of the graph:" + str(node_properties)

  prompt_template = """
    give me a cypher Query to response to this:
    Question: {}
    """.format(question)
  messages = [ {"role": "system", "content": "You are an expert in generating Cypher Query"},
            {"role": "user", "content": prompt_template},
            {"role": "assistant", "content": assistant}
            ]

  response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=messages,
    temperature=0
  )
  print(response.choices[0].message.content)

  ##EXTRACTION DE LA REQUETE CYPHER DE LA REPONSE DE GPT
  contenu_entre_backticks = re.findall(r"```(.*?)```", str(response.choices[0].message.content), re.DOTALL)
  if contenu_entre_backticks:
    for i, contenu in enumerate(contenu_entre_backticks, start=1):
        print(f"Contenu entre les triple backticks #{i}:")

        contenu = contenu.replace("<-", "-").replace("->", "-")
        print(contenu)
        query_result = query_result + str(graph.query(contenu))
  else:
    contenu = response.choices[0].message.content.replace("<-", "-").replace("->", "-")
    query_result =  str(graph.query(contenu))
  ##GENERATION DE LA REPONSE A LA QUESTION
  assistant = "This is the set of facts for the result of the question: \n" + query_result + "\n"
  print(assistant)
  #question2 = "Using a Knowledge Graph, We extract this information:\n"+str(query_result)+".Using only these facts, responds to this question:"+question+ "the facts may contain multiple responses."
  question2 = "Generate a response to this question, using the given set of facts " + question
  print(question2)
  messages =  [{"role" : "system", "content" : "You are an expert in Answering generation using ONLY the given set of facts."},
              {"role": "user", "content": question2 },
              {"role": "assistant", "content": assistant}

              ]
  response = client.chat.completions.create(
      model="gpt-3.5-turbo",
      messages=messages,
      temperature=0
  )
  return ("La réponse à la question est: " + response.choices[0].message.content)

In [34]:
def questionA(client, vector_index, graph, graph2, question):
  answer1 = questionAnswering(question, graph, client)
  answer2 = reponse_question(question,vector_index, client, graph2)

  question2 = "Generate a response to this question, using the given set of facts: \n" + answer1 + "\n" + answer2
  print(question2)


  messages =  [{"role" : "system", "content" : "You are an expert in Answering generation using ONLY the given set of facts."},
              {"role": "user", "content": question2 },
              ]
  response = client.chat.completions.create(
      model="gpt-3.5-turbo",
      messages=messages,
      temperature=0
  )
  return ("La réponse à la question est: " + response.choices[0].message.content)


In [37]:
answer = questionA(client, vector_index, graph, graph2, question)
print(answer)

To find the set of persons who wrote and produced the same movie in Cypher Query, you can use the following query:

```cypher
MATCH (p:Person)-[:Written_by]->(m:Movie)<-[:Produced_by]-(p)
RETURN p.name, m.title
```

This query will match persons who have a relationship of "Written_by" and "Produced_by" with the same movie node, and then return the names of those persons along with the title of the movie they wrote and produced.
Contenu entre les triple backticks #1:
cypher
MATCH (p:Person)-[:Written_by]-(m:Movie)-[:Produced_by]-(p)
RETURN p.name, m.title

This is the set of facts for the result of the question: 
[{'p.name': 'Baz Luhrmann', 'm.title': 'The Great Gatsby'}, {'p.name': 'Adam McKay', 'm.title': "Don't Look Up"}]

Generate a response to this question, using the given set of facts give me the set of persons who wrote and produced the same movie.
HHHHHHHHHHHHI'm sorry, but I couldn't find any specific information in the provided set of facts about the set of persons who wrote 